In [1]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import eq
import wandb

In [12]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/zekai/.netrc


True

In [18]:
catalog = eq.catalogs.ANSS_MultiCatalog(mag_completeness=4.5)

Loading existing catalog from /home/zekai/repos/recast/data/ANSS_MultiCatalog.


In [19]:
def train(config):
    dl_train = catalog.train.get_dataloader()
    dl_val = catalog.val.get_dataloader()
    dl_test = catalog.test.get_dataloader()

    model = eq.models.RecurrentTPP(**config)

    checkpoint = pl_callbacks.ModelCheckpoint(monitor="val_loss")
    early_stopping = pl_callbacks.EarlyStopping(monitor="val_loss", patience=20, min_delta=1e-3)
    progress_bar = pl_callbacks.RichProgressBar()
    trainer = pl.Trainer(accelerator="gpu", max_epochs=200, callbacks=[checkpoint, early_stopping, progress_bar])
    
    try:
        trainer.fit(model, dl_train, dl_val)
    except Exception:
        return float("nan")

    model = eq.models.RecurrentTPP.load_from_checkpoint(checkpoint.best_model_path)
    return trainer.test(model, dl_test)[0]["test_loss"]


In [20]:
res = train({"learning_rate":1e-4})

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

...

`Trainer.fit` stopped: `max_epochs=200` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/home/zekai/miniconda3/envs/eq/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.16295114159584045    │
│      total_test_loss      │    0.16295114159584045    │
└───────────────────────────┴───────────────────────────┘